In [ ]:
# # Descomenten si todavía no instalaron el cliente
# !pip install --upgrade watson-developer-cloud
# !pip install --upgrade watson-machine-learning-client

### Cargando el Dataset
Importamos el dataset de review de películas que utilizamos para la entrega número 5.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
import numpy as np
from sklearn.datasets import load_files
# Acá deben completar con la dirección donde tienen el dataset de películas que usamos para la entrega 5
moviedir = r'./Entregas Resueltas/Entrega 5/dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

Recordemos que este dataset consistía de un diccionario donde la key `data` contenía el texto de la review, y la key `target` tenía un 1 si ese review fue positivo y un 0 si fue negativo.

In [ ]:
movie_reviews.data[0]

In [ ]:
movie_reviews.target

### Armando el pipeline
Primero, antes de definir nuestro workflow de trabajo, vamos a separar en Train y Test nuestros datos.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

Ahora sí, vamos a definir un workflow muy simple donde primero vectorizamos el texto, aplicamos una transofrmación tf-idf y luego clasificamos con un SVM.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.svm import LinearSVC

vect = CountVectorizer()
tfidf = TfidfTransformer()
clf = LinearSVC()

Ahora ponemos estos dos pasos adentro de un único objeto, una instancia de un objeto Pipeline, al que llamamos `pipeline`. Al igual que los otros modelos, también cuenta con las funciones `fit` y `predict`.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

pipeline = Pipeline([('vect',vect),('tfidf',tfidf),('clf',clf)])

In [ ]:
# Podríamos fitear y usar el modelo para predecir de manera local corriendo la siguiente linea.
# Pero no es la idea, vamos a correrlo desde el servidor.
pipeline2 = Pipeline([('vect',vect),('tfidf',tfidf),('clf',clf)])
pipeline2.fit(X_train, y_train)
pipeline2.predict(X_test)

IBM Watson

In [ ]:
# Descomentá la siguiente linea para hacer la instalación
#!pip install watson-machine-learning-client --upgrade

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Para usar Watson Machine Learning API es necesario crear un servicio desde:
https://console.bluemix.net/catalog/services/machine-learning.

Una vez creado ir a la lista de la izquierda a `Service Credentials` y crear nuevas credenciales. 

Para ver las credenciales hay que tocar en `View Credentials`. 
De allí copiar los datos necesarios para completar el código de la siguiente celda

In [ ]:
wml_credentials={
# Copien y peguen sus credenciales
}

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

Creamos los datos del modelo a publicar

In [ ]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Yo", 
               client.repository.ModelMetaNames.NAME: "Reviews classification nuevo",
               client.repository.DefinitionMetaNames.RUNTIME_NAME: 'python',
               client.repository.DefinitionMetaNames.RUNTIME_VERSION: '3.6'}

# Si quieren usar tensorflow, pueden especificar la versión como:
#client.repository.DefinitionMetaNames.NAME: 'my_training_definition',
#client.repository.DefinitionMetaNames.FRAMEWORK_NAME: 'tensorflow',

Subimos el modelo a la nube con `client.repository.store_model`


In [ ]:
published_model = client.repository.store_model(model=pipeline, 
                                                meta_props=model_props, 
                                                training_data=X_train, 
                                                training_target=y_train)


Guardamos el uid del modelo

In [ ]:
# Tomamos el id con el cual nos vamos a referir al modelo
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [ ]:
models_details = client.repository.list_models()

Ahora podemos cargar el modelo a partir de su `uid`

In [ ]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

Mostramos el `classification_report`

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))